# Python: csv to postgres

In [1]:
# Installing packages
!pip install pandas psycopg2 logging python-dotenv requests


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: D:\DataEng\projects\pythonLabs\csv_to_postgres\.csv_to_postgres\Scripts\python.exe -m pip install --upgrade pip


In [2]:
import os
import logging
import psycopg2
import traceback
import urllib.request
from dotenv import load_dotenv

logging.basicConfig(
    level=logging.INFO, format="%(asctime)s:%(funcName)s:%(levelname)s:%(message)s"
)

def conn_postgres():
    try:
        # Load python-dotenv
        load_dotenv()

        # Postgres Envs
        postgres_host = os.environ.get('postgres_host')
        postgres_database = os.environ.get('postgres_database')
        postgres_user = os.environ.get('postgres_user')
        postgres_password = os.environ.get('postgres_password')
        postgres_port = os.environ.get('postgres_port')

        conn = psycopg2.connect(
            host=postgres_host,
            database=postgres_database,
            user=postgres_user,
            password=postgres_password,
            port=postgres_port
        )

        logging.info('Postgres server connection is successful')

        return conn

    except Exception as e:
        traceback.print_exc()
        logging.error("Couldn't create the Postgres connection")
        return None


connection = conn_postgres()

2023-11-17 15:07:24,156:conn_postgres:INFO:Postgres server connection is successful


In [3]:
"""
Downloads the csv file from the URL. Creates a new table in the Postgres server.
Reads the file as a dataframe and inserts each record to the Postgres table. 
"""
url = "https://raw.githubusercontent.com/marcelopachione/pythonLabs/main/datasets/churn_modelling.csv"
dest_folder = os.environ.get("dest_folder")
destination_path = f"{dest_folder}/churn_modelling.csv"


def download_file_from_url(url: str, dest_folder: str):
    """
    Download a file from a specific URL and download to the local direcory
    """
    if not os.path.exists(str(dest_folder)):
        os.makedirs(str(dest_folder))  # create folder if it does not exist

    try:
        urllib.request.urlretrieve(url, destination_path)
        logging.info(
            f"csv file downloaded successfully to the working directory to {destination_path}"
        )
    except Exception as e:
        logging.error(f"Error while downloading the csv file due to: {e}")
        traceback.print_exc()


download_file_from_url(url, dest_folder)

2023-11-17 15:07:24,289:download_file_from_url:INFO:csv file downloaded successfully to the working directory to ../../datasets/churn_modelling.csv


In [108]:
def create_postgres_table():
    """
    Create the Postgres table with a desired schema
    """
    try:
        connection = conn_postgres()

        if connection:
            cur = connection.cursor()
            
            cur.execute("""CREATE TABLE IF NOT EXISTS churn_modelling (RowNumber INTEGER PRIMARY KEY, CustomerId INTEGER, 
            Surname VARCHAR(50), CreditScore INTEGER, Geography VARCHAR(50), Gender VARCHAR(20), Age INTEGER, 
            Tenure INTEGER, Balance FLOAT, NumOfProducts INTEGER, HasCrCard INTEGER, IsActiveMember INTEGER, EstimatedSalary FLOAT, Exited INTEGER)""")
            
            logging.info('New table churn_modelling created successfully on the Postgres server')

    except Exception as e:
        logging.warning('Check if the table churn_modelling exists. Error: {}'.format(e))
    
    finally:
        if connection:
            connection.close()


create_postgres_table()


2023-11-17 15:27:47,583:conn_postgres:INFO:Postgres server connection is successful


2023-11-17 15:27:47,634:create_postgres_table:INFO:New table churn_modelling created successfully on the Postgres server
